In [107]:
# Převede rozsekané XML s Českou národní bibliografií na JSONy, se kterými se už dá pracovat např. v pandas.

import os
import json
from collections import defaultdict
import pymarc

vsechny_sloupce = []

def slovnik_opakovanych(seznam): 
    
    # Aby byly na každém řádku stejně dlouhé seznamy vícenásobných záznamů a šlo např. explodovat jména a role autorstva současně.
    # Ano, každý řádek to prochází třikrát, šlo by to zefektivnit na dva průchody, serepes.

    slovnik = {}
    klice = []
    opakovane_klice = []
    
    for p in seznam:
        if list(p.keys())[0] not in klice:
            klice.append(list(p.keys())[0])
        else:
            opakovane_klice.append(list(p.keys())[0])
    opakovane_klice = list(set(opakovane_klice))
    for p in seznam:
        if list(p.keys())[0] in opakovane_klice:
            pismenka = []
            if 'subfields' in p[list(p.keys())[0]]:
                for o in p[list(p.keys())[0]]['subfields']:
                    pismenka.append(list(o.keys())[0])
            if len(pismenka) > 0:
                if list(p.keys())[0] not in slovnik:
                    slovnik[list(p.keys())[0]] = set()            
                slovnik[list(p.keys())[0]].update(pismenka)
    return slovnik

def na_plochy_slovnik(xml_soubor):

    global vsechny_sloupce

    zaznamy = pymarc.marcxml.parse_xml_to_array(xml_soubor)

    zaznamy = [z for z in zaznamy if "ck8806699" in str(z)]

    print(len(zaznamy))
    
    hesla = []

    for zaznam in zaznamy:

        print(zaznam)
        
        zaznam = json.loads(zaznam.as_json())['fields']

        print(zaznam)
        
        heslo = defaultdict(list)

        doplneni = slovnik_opakovanych(zaznam)

        if "akou barvu má domov" in str(zaznam):
            print(zaznam)
        
        for z in zaznam:
            
            for radek, hodnota in z.items():
                if isinstance(hodnota, str):
                    heslo[radek].append(hodnota)
                elif isinstance(hodnota, dict):
                    for podradek, podhodnota in hodnota.items():
                        if isinstance(podhodnota, str) and podhodnota.strip():
                            heslo[f"{radek}_{podradek}"].append(podhodnota)
                        elif isinstance(podhodnota, list):
                            print(podhodnota)
                            mistni_pismena = []
                            for posledni_uroven in podhodnota:
                                mistni_pismena.append(list(posledni_uroven.keys())[0])
                            print(mistni_pismena)
                            for posledni_uroven in podhodnota:
                                print(posledni_uroven)
                                for posledni_radek, posledni_hodnota in posledni_uroven.items():
                                    mistni_pismena.remove(posledni_radek)
                                    print(mistni_pismena)
                                    if (posledni_radek not in mistni_pismena) or ("700" not in radek):
                                        heslo[f"{radek}_{posledni_radek}"].append(posledni_hodnota)
                            if radek in list(doplneni.keys()):
                                cotuje = [list(podhodnoty.keys())[0] for podhodnoty in podhodnota]
                                for pismenecko in doplneni[radek]:
                                    if pismenecko not in cotuje:
                                        heslo[f"{radek}_{pismenecko}"].append(None)

        print(heslo)
        
        hesla.append(heslo)

        vsechny_sloupce = list(set(vsechny_sloupce + list(heslo.keys())))

    return hesla

vsechna_xml = [f for f in os.listdir('downloads/cnb.xml') if (f[0:3] == 'cnb') and ('_' in f)]

print(f"""{len(vsechna_xml)} souborů XML ve složce. (Počítáme jen ty s podtržítkem.)""")

kam_s_tim = "data_raw/cnb"

if not os.path.exists(kam_s_tim):
    os.makedirs(kam_s_tim)

62 souborů XML ve složce. (Počítáme jen ty s podtržítkem.)


In [109]:
prectene = na_plochy_slovnik(os.path.join('downloads/cnb.xml', 'cnb_002.xml'))

1
=LDR       nam a22        4500
=001  ck8806699
=003  CZ\PrNK
=005  20140815084437.0
=007  tu
=008  881212s1988\\\\xr\\\\\\\\\\\u0|0\|\cze
=015  \\$acnb000041544
=020  \\$q(Váz.) :$c95 Kčs
=035  \\$a(OCoLC)39403690
=040  \\$aABA001$bcze$dABA001
=041  0\$acze$bger$brus
=080  \\$a77.04(437)"1860/198"(084.12)$2undef
=245  00$aJakou barvu má domov :$bčes. a slov. krajina očima fotografů v proměnnách století /$cobr. část sest. a doslov naps. Vladimír Remeš, Daniela Mrázková ; prův. text Libuše Palečková ; úvod Kornel Vavrinčík ; typografie Milan Kopřiva
=250  \\$a1. vyd.
=260  \\$aPraha :$bMladá fronta ;$aBratislava :$bSmena,$c1988$e(Praha :$fSvoboda)
=300  \\$a160 s. :$bbarev. il. ;$c34 cm
=500  \\$aÚdaje o fotografech
=500  \\$aFot. na tit. listu a frontispisu
=500  \\$a20000 výt.
=546  \\$aRus. a něm. resumé
=655  \7$afotografické publikace$7fd132276$2czenas
=700  1\$aVavrinčík, Kornel$7jk01141778$4aui
=700  1\$aMrázková, Daniela,$d1942-$7jk01082289$4com$4aft
=700  1\$aPalečková, Libuše

In [29]:
prectene

[]

In [5]:
    with open(os.path.join(kam_s_tim,'cnb_002.json'), 'w+', encoding='utf-8') as vystup:
        vystup.write(json.dumps(prectene))